In [1]:
# Author: Ciarán O'Brien & Chloe Doyle
# Lecture: Svetlana Hensman
# Submitted: 07/12/18
# This file is in response to the second assignment as set out per the classification goal


In [2]:
# Boiler plate imports
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
import pandas as pd
import re
import cardinality
import statistics
from math import floor
from statistics import mean, median
from collections import Counter
from sympy import pretty_print as pp,latex

In [85]:
data = pd.read_csv("data/trainingset.txt",sep='\,',header=None, encoding='utf-8')

c:\users\ciaran\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [89]:
data.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,TR1,44,"""JobCat9""","""single""","""secondary""","""no""",29,"""yes""","""no""","""unknown""",5,"""may""",0,1,-1,0,"""unknown""","""TypeA"""
1,TR2,31,"""JobCat4""","""married""","""secondary""","""no""",2,"""yes""","""yes""","""unknown""",5,"""may""",0,1,-1,0,"""unknown""","""TypeA"""
2,TR3,42,"""JobCat4""","""divorced""","""tertiary""","""yes""",2,"""yes""","""no""","""unknown""",5,"""may""",0,1,-1,0,"""unknown""","""TypeA"""
3,TR4,58,"""JobCat2""","""married""","""primary""","""no""",121,"""yes""","""no""","""unknown""",5,"""may""",0,1,-1,0,"""unknown""","""TypeA"""
4,TR5,43,"""JobCat9""","""single""","""secondary""","""no""",593,"""yes""","""no""","""unknown""",5,"""may""",0,1,-1,0,"""unknown""","""TypeA"""


In [90]:
# with open('data/datadescription.txt', 'r') as myfile:
#     filedata=myfile.read().replace('\n', '')

In [91]:
# expression = re.compile("((100)|[1-9]\d? - \b[^\d\W]+\b)")
# regx = '(100)|[1-9]\d? - \b[^\d\W]+\b'


# match = re.findall(regx,filedata)
# print(match)
#     #print(re.search(expression,str(dataFeatures[rows])))

In [92]:
data.columns = ["id","age","job","marital","education","default","balance","housing","loan","contact","day","month","duration","campaign","pdays","previous","poutcome","y"]

In [93]:
data.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,TR1,44,"""JobCat9""","""single""","""secondary""","""no""",29,"""yes""","""no""","""unknown""",5,"""may""",0,1,-1,0,"""unknown""","""TypeA"""
1,TR2,31,"""JobCat4""","""married""","""secondary""","""no""",2,"""yes""","""yes""","""unknown""",5,"""may""",0,1,-1,0,"""unknown""","""TypeA"""
2,TR3,42,"""JobCat4""","""divorced""","""tertiary""","""yes""",2,"""yes""","""no""","""unknown""",5,"""may""",0,1,-1,0,"""unknown""","""TypeA"""
3,TR4,58,"""JobCat2""","""married""","""primary""","""no""",121,"""yes""","""no""","""unknown""",5,"""may""",0,1,-1,0,"""unknown""","""TypeA"""
4,TR5,43,"""JobCat9""","""single""","""secondary""","""no""",593,"""yes""","""no""","""unknown""",5,"""may""",0,1,-1,0,"""unknown""","""TypeA"""
